# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on textual data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imported_data.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
# each sequence represents a sentence.

25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like
# This is a sentence consists of 30 words, and each number represents a word.

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer [Embedding will make similar words have vectors close to each other] 
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe) [So it would be specific to our data context]
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Kernel_initializer is going to be the weights for the inputs.
- recurrent_initializer is going to be the initialized weights for those state layers.
- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
print(x_train.shape[0])
print(x_train.shape[1])

25000
30


In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50 #we will take the current integers (words) and given their context, we will come up with an embedding where it's going to transfer each integer into an embedded vector thats of dimension 50.
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:])) # Shape of a one single vector 

model_rnn.add(Dense(1, activation='sigmoid')) # so we can have an output of 1/0 [positive or negative]

2022-08-03 16:54:26.644203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 14s 16ms/step - loss: 0.6546 - accuracy: 0.6262 - val_loss: 0.5935 - val_accuracy: 0.6967
Epoch 2/10
782/782 [==============================] - 11s 15ms/step - loss: 0.5470 - accuracy: 0.7294 - val_loss: 0.5316 - val_accuracy: 0.7318
Epoch 3/10
782/782 [==============================] - 11s 15ms/step - loss: 0.4834 - accuracy: 0.7681 - val_loss: 0.4974 - val_accuracy: 0.7526
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4430 - accuracy: 0.7942 - val_loss: 0.4794 - val_accuracy: 0.7662
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4152 - accuracy: 0.8108 - val_loss: 0.4635 - val_accuracy: 0.7757
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3951 - accuracy: 0.8229 - val_loss: 0.4534 - val_accuracy: 0.7824
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3809 - accuracy: 0.8289 - val_loss: 0.4518 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 3s 4ms/step - loss: 0.4462 - accuracy: 0.7917
Test score: 0.44624343514442444
Test accuracy: 0.7917199730873108


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imported_data.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 23s 28ms/step - loss: 0.6090 - accuracy: 0.6609 - val_loss: 0.5167 - val_accuracy: 0.7504
Epoch 2/10
782/782 [==============================] - 21s 27ms/step - loss: 0.4578 - accuracy: 0.7870 - val_loss: 0.5235 - val_accuracy: 0.7324
Epoch 3/10
782/782 [==============================] - 28s 36ms/step - loss: 0.3890 - accuracy: 0.8274 - val_loss: 0.4128 - val_accuracy: 0.8144
Epoch 4/10
782/782 [==============================] - 27s 35ms/step - loss: 0.3476 - accuracy: 0.8509 - val_loss: 0.3953 - val_accuracy: 0.8229
Epoch 5/10
782/782 [==============================] - 28s 36ms/step - loss: 0.3194 - accuracy: 0.8660 - val_loss: 0.3857 - val_accuracy: 0.8294
Epoch 6/10
782/782 [==============================] - 31s 40ms/step - loss: 0.2986 - accuracy: 0.8742 - val_loss: 0.3810 - val_accuracy: 0.8296
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.2831 - accuracy: 0.8816 - val_loss: 0.3669 - val_accuracy:

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imported_data.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 24s 29ms/step - loss: 0.6503 - accuracy: 0.6179 - val_loss: 0.5783 - val_accuracy: 0.7028
Epoch 2/10
782/782 [==============================] - 23s 29ms/step - loss: 0.5200 - accuracy: 0.7473 - val_loss: 0.5012 - val_accuracy: 0.7572
Epoch 3/10
782/782 [==============================] - 23s 29ms/step - loss: 0.4602 - accuracy: 0.7812 - val_loss: 0.4686 - val_accuracy: 0.7712
Epoch 4/10
782/782 [==============================] - 23s 29ms/step - loss: 0.4186 - accuracy: 0.8103 - val_loss: 0.4761 - val_accuracy: 0.7809
Epoch 5/10
782/782 [==============================] - 23s 29ms/step - loss: 0.3853 - accuracy: 0.8307 - val_loss: 0.4210 - val_accuracy: 0.8069
Epoch 6/10
782/782 [==============================] - 23s 30ms/step - loss: 0.3643 - accuracy: 0.8411 - val_loss: 0.4074 - val_accuracy: 0.8115
Epoch 7/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3505 - accuracy: 0.8481 - val_loss: 0.4014 - val_accuracy:

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 22s 29ms/step - loss: 0.3150 - accuracy: 0.8649 - val_loss: 0.3802 - val_accuracy: 0.8291
Epoch 2/10
782/782 [==============================] - 23s 29ms/step - loss: 0.3086 - accuracy: 0.8707 - val_loss: 0.3734 - val_accuracy: 0.8325
Epoch 3/10
782/782 [==============================] - 23s 29ms/step - loss: 0.3026 - accuracy: 0.8724 - val_loss: 0.3693 - val_accuracy: 0.8340
Epoch 4/10
782/782 [==============================] - 23s 30ms/step - loss: 0.2982 - accuracy: 0.8740 - val_loss: 0.3711 - val_accuracy: 0.8339
Epoch 5/10
782/782 [==============================] - 23s 30ms/step - loss: 0.2939 - accuracy: 0.8756 - val_loss: 0.3674 - val_accuracy: 0.8365
Epoch 6/10
782/782 [==============================] - 23s 30ms/step - loss: 0.2897 - accuracy: 0.8769 - val_loss: 0.3628 - val_accuracy: 0.8389
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.2865 - accuracy: 0.8794 - val_loss: 0.3639 - val_accuracy:

### Trying a Different Network

In [21]:
max_features = 1000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 40  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imported_data.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [22]:
rnn_hidden_dim = 8
word_embedding_dim = 25
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [24]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))

Epoch 1/15
782/782 [==============================] - 18s 21ms/step - loss: 0.6650 - accuracy: 0.6013 - val_loss: 0.6243 - val_accuracy: 0.6560
Epoch 2/15
782/782 [==============================] - 17s 22ms/step - loss: 0.5890 - accuracy: 0.6891 - val_loss: 0.5616 - val_accuracy: 0.7115
Epoch 3/15
782/782 [==============================] - 18s 23ms/step - loss: 0.5453 - accuracy: 0.7224 - val_loss: 0.5463 - val_accuracy: 0.7178
Epoch 4/15
782/782 [==============================] - 19s 24ms/step - loss: 0.5162 - accuracy: 0.7411 - val_loss: 0.5070 - val_accuracy: 0.7536
Epoch 5/15
782/782 [==============================] - 17s 22ms/step - loss: 0.4944 - accuracy: 0.7589 - val_loss: 0.4882 - val_accuracy: 0.7652
Epoch 6/15
782/782 [==============================] - 19s 24ms/step - loss: 0.4777 - accuracy: 0.7708 - val_loss: 0.4751 - val_accuracy: 0.7700
Epoch 7/15
782/782 [==============================] - 17s 22ms/step - loss: 0.4658 - accuracy: 0.7778 - val_loss: 0.4767 - val_accuracy:

In [25]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 4s 5ms/step - loss: 0.4532 - accuracy: 0.7852
Test score: 0.4532153904438019
Test accuracy: 0.7851600050926208


---
### Machine Learning Foundation (C) 2020 IBM Corporation